<a href="https://colab.research.google.com/github/Redcoder815/Deep_Learning_TensorFlow/blob/main/12CustomLayers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

Layers without Parameters

In [2]:
class CenteredLayer(tf.keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, X):
        return X - tf.reduce_mean(X)

In [3]:
layer = CenteredLayer()
layer(tf.constant([1.0, 2, 3, 4, 5]))

<tf.Tensor: shape=(5,), dtype=float32, numpy=array([-2., -1.,  0.,  1.,  2.], dtype=float32)>

In [4]:
net = tf.keras.Sequential([tf.keras.layers.Dense(128), CenteredLayer()])

In [5]:
Y = net(tf.random.uniform((4, 8)))
tf.reduce_mean(Y)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

Layers with Parameters

In [6]:
class MyDense(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias
        return tf.nn.relu(linear)

In [7]:
dense = MyDense(3)
dense(tf.random.uniform((2, 5)))
dense.get_weights()

[array([[-0.00204242,  0.15273716, -0.0009473 ],
        [-0.03889515,  0.01813225, -0.00726042],
        [-0.06612141,  0.00312592, -0.01205603],
        [-0.02460948,  0.04594931, -0.03052509],
        [-0.00405091,  0.02957712,  0.04582574]], dtype=float32),
 array([0., 0., 0.], dtype=float32)]

In [8]:
dense(tf.random.uniform((2, 5)))

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.        , 0.08322311, 0.01207601],
       [0.        , 0.07136088, 0.01157495]], dtype=float32)>

The output shape (2, 1) is due to the structure of the net model and the input it receives. Let's break it down:

Input: The model receives an input tensor with shape (2, 64). The 2 represents the batch size (number of samples), and 64 represents the number of input features.
First MyDense(8) layer:
This layer has 8 units, meaning it will transform the input features into 8 output features.
The matrix multiplication tf.matmul(X, self.weight) will multiply a (2, 64) input with a (64, 8) weight matrix, resulting in an output of shape (2, 8).
The ReLU activation function then applies element-wise, preserving the (2, 8) shape.
Second MyDense(1) layer:
This layer takes the output of the first layer (shape (2, 8)) as its input.
It has 1 unit, meaning it will transform the 8 input features from the previous layer into 1 output feature.
The matrix multiplication tf.matmul(X, self.weight) will multiply a (2, 8) input with an (8, 1) weight matrix, resulting in an output of shape (2, 1).
Again, the ReLU activation preserves this shape.
So, the final output of the net model is (2, 1), which corresponds to 2 rows (for the 2 input samples) and 1 column (for the single output feature from the last dense layer).

In [9]:
net = tf.keras.models.Sequential([MyDense(8), MyDense(1)])
net(tf.random.uniform((2, 64)))

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [0.]], dtype=float32)>